In [77]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow
from tensorflow import keras
from tensorflow.keras.utils import img_to_array,array_to_img
from tensorflow.python.keras.utils.data_utils import Sequence 
from tensorflow.keras.utils import Sequence

In [78]:
print(tensorflow.__version__)

2.9.1


# fashion_mnist 

### Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits.

The original MNIST dataset contains a lot of handwritten digits. Members of the AI/ML/Data Science community love this dataset and use it as a benchmark to validate their algorithms. In fact, MNIST is often the first dataset researchers try. "If it doesn't work on MNIST, it won't work at all", they said. "Well, if it does work on MNIST, it may still fail on others."

Zalando seeks to replace the original MNIST dataset

### Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255. The training and test data sets have 785 columns. The first column consists of the class labels (see above), and represents the article of clothing. The rest of the columns contain the pixel-values of the associated image.

To locate a pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27. The pixel is located on row i and column j of a 28 x 28 matrix.
For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

### Labels

Each training and test example is assigned to one of the following labels:

0 T-shirt/top

1 Trouser

2 Pullover

3 Dress

4 Coat

5 Sandal

6 Shirt

7 Sneaker

8 Bag

9 Ankle boot

In [97]:
from keras.datasets import mnist


In [98]:
ds=keras.datasets.fashion_mnist

In [99]:
(train_images,train_labels),(test_images,test_labels)=ds.load_data()

In [100]:
len(train_images)

60000

In [101]:
train_images=train_images/255.0
test_images=test_images/255.0

In [102]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [103]:
train_images.shape

(60000, 28, 28, 1)

In [104]:
def bulid_model(hp):
    model=keras.Sequential([keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                                kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
                                                activation='relu',
                                                input_shape=(28,28,1)),
                            keras.layers.Conv2D(filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
                                                kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
                                                activation='relu'),
                            keras.layers.Flatten(),
                            keras.layers.Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), 
                                               activation="relu"),
                            keras.layers.Dense(10,activation='softmax')
                           ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

                             
    
    

In [105]:
 from kerastuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [108]:
tuner_search=RandomSearch(bulid_model,objective='val_accuracy',max_trials=5,directory='output',project_name='mnist fashion')

In [ ]:
  tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 2 Complete [00h 45m 47s]
val_accuracy: 0.8491666913032532

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 01h 06m 41s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
96                |64                |conv_1_filter
3                 |5                 |conv_1_kernel
3                 |5                 |conv_2_kernel
64                |96                |units
0.0001            |0.001             |learning_rate

Epoch 1/3
 750/1688 [============>.................] - ETA: 1:22 - loss: 0.5740 - accuracy: 0.7970